In [1]:
import os
import re
import json

from tqdm import tqdm

In [2]:
from langchain_community.llms import Ollama
# llm =  Ollama(model="deepseek-r1:32b", temperature=0)

## 加载本地的GDP数据集

In [3]:
def load_json(file, encoding="utf-8"):
    with open(file, "r", encoding=encoding) as f:
        return json.load(f)

def save_to_json(obj, file, encoding="utf-8"):
    parent_fold = os.path.dirname(file)
    os.makedirs(parent_fold, exist_ok=True)
    with open(file, "w", encoding=encoding) as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

In [4]:
dataset = load_json("../data/长沙_gdp.json")

In [5]:
next(iter(dataset.items()))

('长沙市2003.txt',
 {'text': [], 'label': '<not found>', 'year': '2003', 'city': '长沙'})

In [6]:
PROMPT_TEMPLATE = """请你阅读长沙市的政府工作报告文本节选，抽取其中GDP具体的值。
首先会在前面的文中提到全年（表述可能为：全年、今年、去年）的国民生产总值的具体数据，然后会在后面的文本展望未来预期的国民生产总值。
如果文中出现了多个对GDP值的表述请你输出前面的。不要返回政府对未来预期的GDP的值。

文本节选如下：
{{text}}

返回json格式:
```json
{
    "reason":"说明理由",
    "GDP": "GDP值"
}
```
GDP值为数字，不需要单位，若文中未提到GDP值，请填写"<not found>"
"""

In [8]:
# # Please install OpenAI SDK first: `pip3 install openai`

# from openai import OpenAI

# client = OpenAI(base_url="http://localhost:11434/v1")

# response = client.chat.completions.create(
#     model="deepseek-r1:32b",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant"},
#         {"role": "user", "content": "Who are you?"},
#     ],
#     stream=False
# )

## 解析大模型抽取结果

In [6]:
dataset['长沙市2004.txt']

{'text': ['全年完成国内生产总值９２８.２亿元（预计数，下同），增长１４％；全市财政总收入达１００.０２亿元，增长３２.５％，完成地方财政收入７５亿元，增长３６.４％；一、二、三产业分别增长４.８％、２０.３％和１０.５％，三次产业结构为８.９∶４２.５∶４８.６；工业完成增加值２６８.５亿元，规模工业实现利润３８亿元，均创历史新高；在稳定粮食生产的前提下，全市增加经济作物种植面积１７万亩，养殖业产值占农业总产值比重达４６.３％；乡镇企业完成产值８４０亿元，增长１５％；高新技术产业实现产值３９１亿元，增长２５％；社会消费品零售总额４５２亿元，增长１２.６％。２００３年我市经济发展呈现的良好态势，标志着全市经济运行进入了新一轮发展增长期！',
  '今年政府工作的总体要求是：深入贯彻落实党的十六大和十六届三中全会精神，紧紧围绕全面建设小康社会的总体目标，坚持以人为本，树立全面、协调、可持续的发展观，深化改革，扩大开放，统筹发展，维护稳定，进一步完善社会主义市场经济体制，加快推进工业化、城市化、农业产业化进程，努力提高人民生活水平，实现国民经济持续快速协调健康发展和社会全面进步。今年全市经济和社会发展的主要预期目标是：国内生产总值增长１２％以上；全社会固定资产投资总额增长１８％以上；社会消费品零售总额增长１３％以上；地方财政收入增长１５％以上；城市居民和农村居民人均可支配收入分别增长９％和７％；城镇登记失业率控制在４％以内；人口自然增长率控制在５‰以内。'],
 'label': '928.2',
 'year': '2004',
 'city': '长沙'}

In [9]:
print("\n".join( dataset["长沙市2004.txt"]["text"]))

全年完成国内生产总值９２８.２亿元（预计数，下同），增长１４％；全市财政总收入达１００.０２亿元，增长３２.５％，完成地方财政收入７５亿元，增长３６.４％；一、二、三产业分别增长４.８％、２０.３％和１０.５％，三次产业结构为８.９∶４２.５∶４８.６；工业完成增加值２６８.５亿元，规模工业实现利润３８亿元，均创历史新高；在稳定粮食生产的前提下，全市增加经济作物种植面积１７万亩，养殖业产值占农业总产值比重达４６.３％；乡镇企业完成产值８４０亿元，增长１５％；高新技术产业实现产值３９１亿元，增长２５％；社会消费品零售总额４５２亿元，增长１２.６％。２００３年我市经济发展呈现的良好态势，标志着全市经济运行进入了新一轮发展增长期！
今年政府工作的总体要求是：深入贯彻落实党的十六大和十六届三中全会精神，紧紧围绕全面建设小康社会的总体目标，坚持以人为本，树立全面、协调、可持续的发展观，深化改革，扩大开放，统筹发展，维护稳定，进一步完善社会主义市场经济体制，加快推进工业化、城市化、农业产业化进程，努力提高人民生活水平，实现国民经济持续快速协调健康发展和社会全面进步。今年全市经济和社会发展的主要预期目标是：国内生产总值增长１２％以上；全社会固定资产投资总额增长１８％以上；社会消费品零售总额增长１３％以上；地方财政收入增长１５％以上；城市居民和农村居民人均可支配收入分别增长９％和７％；城镇登记失业率控制在４％以内；人口自然增长率控制在５‰以内。


In [11]:
print(PROMPT_TEMPLATE.replace("{{text}}", "\n".join( dataset["长沙市2004.txt"]["text"])))

请你阅读长沙市的政府工作报告文本节选，抽取其中GDP具体的值。
首先会在前面的文中提到全年（表述可能为：全年、今年、去年）的国民生产总值的具体数据，然后会在后面的文本展望未来预期的国民生产总值。
如果文中出现了多个对GDP值的表述请你输出前面的。不要返回政府对未来预期的GDP的值。

文本节选如下：
全年完成国内生产总值９２８.２亿元（预计数，下同），增长１４％；全市财政总收入达１００.０２亿元，增长３２.５％，完成地方财政收入７５亿元，增长３６.４％；一、二、三产业分别增长４.８％、２０.３％和１０.５％，三次产业结构为８.９∶４２.５∶４８.６；工业完成增加值２６８.５亿元，规模工业实现利润３８亿元，均创历史新高；在稳定粮食生产的前提下，全市增加经济作物种植面积１７万亩，养殖业产值占农业总产值比重达４６.３％；乡镇企业完成产值８４０亿元，增长１５％；高新技术产业实现产值３９１亿元，增长２５％；社会消费品零售总额４５２亿元，增长１２.６％。２００３年我市经济发展呈现的良好态势，标志着全市经济运行进入了新一轮发展增长期！
今年政府工作的总体要求是：深入贯彻落实党的十六大和十六届三中全会精神，紧紧围绕全面建设小康社会的总体目标，坚持以人为本，树立全面、协调、可持续的发展观，深化改革，扩大开放，统筹发展，维护稳定，进一步完善社会主义市场经济体制，加快推进工业化、城市化、农业产业化进程，努力提高人民生活水平，实现国民经济持续快速协调健康发展和社会全面进步。今年全市经济和社会发展的主要预期目标是：国内生产总值增长１２％以上；全社会固定资产投资总额增长１８％以上；社会消费品零售总额增长１３％以上；地方财政收入增长１５％以上；城市居民和农村居民人均可支配收入分别增长９％和７％；城镇登记失业率控制在４％以内；人口自然增长率控制在５‰以内。

返回json格式:
```json
{
    "reason":"说明理由",
    "GDP": "GDP值"
}
```
GDP值为数字，不需要单位，若文中未提到GDP值，请填写"<not found>"



In [ ]:
def llm_pred(llm, dataset, output_file):
    pred_dataset = []
    for item in tqdm(dataset.values()):
        if item["text"] == []:
            continue
        text = "\n".join(item["text"])
        prompt = PROMPT_TEMPLATE.replace("{{text}}", text)
        item["prompt"] = prompt
        llm_answer = llm.invoke(prompt)
        item["llm_answer"] = llm_answer
        llm_response_text = re.search(
            r"```json\n(.*)\n```", llm_answer, re.DOTALL
        ).group(1)
        reponse_d = json.loads(llm_response_text)
        pred_label = reponse_d.get("GDP", "error")
        item["pred_label"] = pred_label
        pred_dataset.append(item)
        
    save_to_json(pred_dataset, output_file)

In [ ]:
qwen_7b =  Ollama(model="qwen2.5:7b", temperature=0)
llm_pred(qwen_7b, dataset, "output/长沙_gdp_qwen_7b.json")

100%|██████████| 21/21 [00:59<00:00,  2.81s/it]


In [17]:
deepseek_r1_32b =  Ollama(model="deepseek-r1:32b", temperature=0)
llm_pred(deepseek_r1_32b, dataset, "output/长沙_gdp_deepseek_r1_32b.json")

100%|██████████| 21/21 [04:45<00:00, 13.59s/it]


In [15]:
deepseek_r1_7b = Ollama(model="deepseek-r1:7b", temperature=0)
llm_pred(deepseek_r1_7b, dataset, "output/长沙_gdp_deepseek_r1_7b.json")

100%|██████████| 21/21 [01:43<00:00,  4.92s/it]


In [9]:
qwq_32b = Ollama(model="qwq:32b", temperature=0)
llm_pred(qwq_32b, dataset, "output/长沙_gdp_qwq_32b.json")

100%|██████████| 21/21 [08:34<00:00, 24.50s/it]
